# 데이터 사이언스 최종 과제 
### 컴퓨터공학과 
### 202130422 박연수 

### train.csv   
주어진 훈련 데이터는 store_nbr, family, onpromotion이라는 특징들의 시계열과 목표 변수인 sales로 구성됩니다.   

- store_nbr: 제품이 판매된 매장을 식별합니다.   
- family: 판매된 제품의 종류를 식별합니다.   
- sales: 특정 날짜에 특정 매장에서 제품군의 총 판매량을 나타냅니다. 제품이 분수 단위로 판매될 수 있기 때문에, 판매량은 소수 값일 수도 있습니다(예: 치즈 1.5kg 또는 감자칩 1봉지).   
- onpromotion: 특정 날짜에 특정 매장에서 특정 제품군의 프로모션 중인 아이템 수를 나타냅니다.   

### test.csv
테스트 데이터는 훈련 데이터와 동일한 특징을 가지고 있습니다. 이 파일에 있는 날짜에 대해 목표 판매량(sales)을 예측해야 합니다. 테스트 데이터의 날짜는 훈련 데이터의 마지막 날짜 이후 15일 동안의 날짜를 포함합니다.

### sample_submission.csv
올바른 형식의 샘플 제출 파일입니다.

### stores.csv
도시, 주(state), 유형(type), 클러스터(cluster)를 포함한 매장 메타데이터입니다. 클러스터는 유사한 매장들의 그룹을 나타냅니다.

### oil.csv
일별 유가를 나타냅니다. 훈련 데이터와 테스트 데이터 기간 동안의 유가를 포함합니다. 에콰도르는 석유 의존 국가이므로, 유가 충격에 경제적으로 매우 취약합니다.

### holidays_events.csv
휴일 및 이벤트에 대한 메타데이터를 포함하고 있습니다.

참고: transferred 열에 특히 주의해야 합니다. 공식적으로 휴일이 해당 날짜에 있지만 정부에 의해 다른 날짜로 이동된 경우, transferred 열이 표시됩니다. 이 경우, 해당 날짜는 일반 평일과 비슷합니다. 실제로 축하된 날짜를 찾으려면, type이 Transfer인 행을 찾아보세요. 예를 들어, Independencia de Guayaquil 휴일은 2012-10-09에서 2012-10-12로 이동되었습니다. 이는 2012-10-12에 축하되었음을 의미합니다. Bridge 유형은 휴일에 추가되는 추가 날짜를 나타내며, 자주 Work Day 유형으로 보상됩니다. Work Day는 일반적으로 근무하지 않는 날(예: 토요일)을 나타내며, Bridge를 보상하기 위해 사용됩니다. 추가 휴일은 정규 달력 휴일에 추가되는 날을 의미하며, 예를 들어, 크리스마스 주위에 자주 발생합니다(크리스마스 이브를 휴일로 만드는 경우).
이 정보를 바탕으로 테스트 데이터에 대해 예측을 수행하고, 결과를 제출 파일 형식에 맞추어 제출해야 합니다.

In [1]:
import pandas as pd 

holidays_events = pd.read_csv('data/holidays_events.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')
stores = pd.read_csv('data/stores.csv')
oil = pd.read_csv('data/oil.csv')

test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')
transactions = pd.read_csv('data/transactions.csv') 

In [2]:
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8
